In [ ]:
import os
from typing import Optional
from sqlmodel import Field, SQLModel, create_engine, Session
import pathlib
from IPython.display import display, Markdown
from kowinski.parser.repo_parser import parse_repository
from kowinski.parser.file_parser import analyze_python_files
from kowinski.agents.code_agent import create_analysis_agent, create_code_agent, create_model, create_patch_agent
import pandas as pd 
import sqlite3
from smolagents import load_tool, CodeAgent, HfApiModel, DuckDuckGoSearchTool, OpenAIServerModel, LiteLLMModel, tool
from vllm import LLM
from dotenv import load_dotenv
import base64
from opentelemetry.sdk.trace import TracerProvider
import difflib
from openinference.instrumentation.smolagents import SmolagentsInstrumentor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

load_dotenv()

LANGFUSE_PUBLIC_KEY=os.getenv("LANGFUSE_PUBLIC_KEY")
LANGFUSE_SECRET_KEY=os.getenv("LANGFUSE_SECRET_KEY")
LANGFUSE_AUTH=base64.b64encode(f"{LANGFUSE_PUBLIC_KEY}:{LANGFUSE_SECRET_KEY}".encode()).decode()

os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = "https://cloud.langfuse.com/api/public/otel" # EU data region
# os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = "https://us.cloud.langfuse.com/api/public/otel" # US data region
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"Authorization=Basic {LANGFUSE_AUTH}"


In [3]:
issue_data = pd.read_parquet('../data/data.parquet')
row = issue_data.iloc[0]
repo_path = f'../data/repos/repo__{row["instance_id"]}'

In [4]:
trace_provider = TracerProvider()
trace_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter()))
SmolagentsInstrumentor().instrument(tracer_provider=trace_provider)

In [5]:
model = create_model(
)

# Create patch agent
patch_agent = create_patch_agent(
    model=model,
    db_path='repository.db'
)

task = f"""You are tasked with understanding an issue in the following github repository {row['repo']}.
You will be given a description of the issue and tools to search the codebase.
Your job is to determine, what file or files are causing the issue.
The tools provided will allow you to search the codebase and find the files that are causing the issue.
Determine what is the function or class where this issue is happening
### Issue Description
{row['problem_statement']}"""
# Run the agent

In [6]:
Markdown(patch_agent.system_prompt)

You are an expert software engineer assistant who specializes in analyzing and fixing code issues. Your task is to generate patches to fix GitHub issues in code repositories.

To do so, you have been given access to a set of tools to help you understand code files, analyze issues, and create patches.

To solve the task, you must plan forward to proceed in a series of steps, in a cycle of 'Thought:', 'Code:', and 'Observation:' sequences.

At each step, in the 'Thought:' sequence, you should first explain your reasoning towards understanding the issue and the tools that you want to use.
Then in the 'Code:' sequence, you should write the code in simple Python. The code sequence must end with '<end_code>' sequence.
During each intermediate step, you can use 'print()' to save whatever important information you will then need.
These print outputs will then appear in the 'Observation:' field, which will be available as input for the next step.
In the end you have to return a final answer using the `final_answer` tool.

Here are a few examples using the patch generation tools:

---
Task: "Fix this issue in the file utils/auth.py:

Bug report: Authentication fails when username contains an '@' symbol. The app crashes with the error:

```
Traceback (most recent call last):
  File 'app.py', line 67, in login
    user = authenticate(username, password)
  File 'utils/auth.py', line 42, in authenticate
    parts = username.split('@')
    email = '@'.join(parts[1:])
IndexError: list index out of range
```

The issue is that the code assumes all usernames have an '@' symbol, but they don't."

Thought: I need to investigate the authenticate function in utils/auth.py. The traceback shows that the error happens on line 42, where the code tries to access parts[1:], but parts might be empty if the username doesn't contain an '@' symbol.
Code:
```py
# First, let's look at the file content to understand the context
file_content = get_file_content("utils/auth.py")
print(file_content)
```<end_code>
Observation:
```python
# utils/auth.py
import hashlib
from database import get_user_by_email

def authenticate(username, password):
    """
    Authenticate a user.
    
    Args:
        username: The username to authenticate
        password: The user's password
        
    Returns:
        User object if authentication successful, None otherwise
    """
    # Extract email from username (assuming username is email)
    parts = username.split('@')
    email = '@'.join(parts[1:])
    
    if not email:
        return None
    
    # Get user from database
    user = get_user_by_email(email)
    if not user:
        return None
    
    # Validate password
    hashed_password = hashlib.sha256(password.encode()).hexdigest()
    if user.password != hashed_password:
        return None
    
    return user
```

Thought: Now I see the issue. In the authenticate function, it splits the username by '@' and assumes there's at least one '@' in the username. The issue happens when a username doesn't contain '@'. Let me use the extract_method_context tool to get more context around this function.
Code:
```py
method_context = extract_method_context(file_content, "authenticate")
print(method_context)
```<end_code>
Observation:
{
  "found": true,
  "start_line": 4,
  "end_line": 31,
  "content": "def authenticate(username, password):\n    \"\"\"\n    Authenticate a user.\n    \n    Args:\n        username: The username to authenticate\n        password: The user's password\n        \n    Returns:\n        User object if authentication successful, None otherwise\n    \"\"\"\n    # Extract email from username (assuming username is email)\n    parts = username.split('@')\n    email = '@'.join(parts[1:])\n    \n    if not email:\n        return None\n    \n    # Get user from database\n    user = get_user_by_email(email)\n    if not user:\n        return None\n    \n    # Validate password\n    hashed_password = hashlib.sha256(password.encode()).hexdigest()\n    if user.password != hashed_password:\n        return None\n    \n    return user",
  "context_before": "# utils/auth.py\nimport hashlib\nfrom database import get_user_by_email",
  "context_after": "",
  "full_range": [3, 30]
}

Thought: Now let's identify patterns in the file that might be related to the issue.
Code:
```py
issue_patterns = identify_issue_patterns(file_content, "Bug report: Authentication fails when username contains an '@' symbol. The app crashes with the error:\n\nTraceback (most recent call last):\n  File 'app.py', line 67, in login\n    user = authenticate(username, password)\n  File 'utils/auth.py', line 42, in authenticate\n    parts = username.split('@')\n    email = '@'.join(parts[1:])\nIndexError: list index out of range")
print(issue_patterns)
```<end_code>
Observation:
[
  {
    "type": "function_definition",
    "name": "authenticate",
    "line": 4,
    "content": "def authenticate(username, password):"
  },
  {
    "type": "traceback_line",
    "line": 42,
    "content": "    email = '@'.join(parts[1:])",
    "context": "  File 'utils/auth.py', line 42, in authenticate\n    parts = username.split('@')\n    email = '@'.join(parts[1:])\n"
  }
]

Thought: I've identified the issue. When a username doesn't contain an '@' symbol, the parts list will just have one element, and parts[1:] will be an empty list. When trying to join an empty list with '@', there's no issue, but the resulting email variable will be empty.

The fix should check if the username contains an '@' symbol before trying to extract the email part. If it doesn't, we should handle that case appropriately. Let me create a patch operation to fix this.
Code:
```py
# Define the patch operations
patch_operations = [
  {
    "operation": "replace",
    "start_line": 13,
    "end_line": 15,
    "content": "    # Extract email from username (assuming username is email)\n    if '@' in username:\n        parts = username.split('@')\n        email = '@'.join(parts[1:])\n    else:\n        # If no @ symbol, assume username is the email\n        email = username"
  }
]

# Apply the patch
modified_content = apply_patch_to_content(file_content, patch_operations)

# Generate the diff
diff = generate_diff(file_content, modified_content, "utils/auth.py")
print(diff)
```<end_code>
Observation:
--- a/utils/auth.py
+++ b/utils/auth.py
@@ -10,8 +10,11 @@
     Returns:
         User object if authentication successful, None otherwise
     """
     # Extract email from username (assuming username is email)
-    parts = username.split('@')
-    email = '@'.join(parts[1:])
+    if '@' in username:
+        parts = username.split('@')
+        email = '@'.join(parts[1:])
+    else:
+        # If no @ symbol, assume username is the email
+        email = username
     
     if not email:
         return None

Thought: The diff looks good. I've modified the code to check if '@' is in the username first. If it is, it proceeds with the original logic of extracting the email part. If not, it just assumes the entire username is the email. This will prevent the IndexError when a username doesn't contain an '@' symbol.
Code:
```py
final_answer({
  "file_path": "utils/auth.py",
  "modified_content": modified_content,
  "diff": diff,
  "description": "Fixed the authentication function to handle usernames without an '@' symbol. The original code assumed all usernames would have an '@' and tried to extract the email part by splitting on '@', which causes an IndexError when no '@' is present. The fix adds a check to see if '@' is in the username before trying to split it, and if not, it just uses the entire username as the email."
})
```<end_code>
---

You must carefully analyze the issue and file to understand the root cause of the problem. Consider edge cases, error handling, and test scenarios when designing your patch. Make sure your patch is minimal and focused on fixing the specific issue.

On top of performing computations in the Python code snippets that you create, you only have access to these tools:
- get_folders: Get all unique folder paths in the repository.
    Takes inputs: {}
    Returns an output of type: array
- get_files_in_folder: Get all files in a specific folder.
    Takes inputs: {'folder_path': {'type': 'string', 'description': 'The relative folder path to query.'}}
    Returns an output of type: array
- get_files_by_extension: Get all files with a specific extension.
    Takes inputs: {'extension': {'type': 'string', 'description': "The file extension to filter by (without the dot, e.g., 'py' not '.py')."}}
    Returns an output of type: array
- get_file_by_path: Get file information for a specific file path.
    Takes inputs: {'file_path': {'type': 'string', 'description': 'The relative path to the file, including folder and filename.'}}
    Returns an output of type: object
- get_file_content: Get the content of a file by path.
    Takes inputs: {'file_path': {'type': 'string', 'description': 'The relative path to the file, including folder and filename.'}}
    Returns an output of type: string
- get_entity_at_line: Find the code entity (function, class, or variable) at a specific line in a file.
    Takes inputs: {'file_path': {'type': 'string', 'description': 'The relative path to the file, including folder and filename.'}, 'line_number': {'type': 'integer', 'description': 'The line number to check.'}}
    Returns an output of type: object
- get_function_by_name: Find functions or methods by name, optionally filtering by class and file.
    Takes inputs: {'function_name': {'type': 'string', 'description': 'The name of the function or method to find.'}, 'class_name': {'type': 'string', 'nullable': True, 'description': 'Optional class name to filter by (for methods).'}, 'file_path': {'type': 'string', 'nullable': True, 'description': 'Optional file path to limit the search to a specific file.'}}
    Returns an output of type: array
- get_class_by_name: Find classes by name, optionally filtering by file.
    Takes inputs: {'class_name': {'type': 'string', 'description': 'The name of the class to find.'}, 'file_path': {'type': 'string', 'nullable': True, 'description': 'Optional file path to limit the search to a specific file.'}}
    Returns an output of type: array
- get_class_methods: Get all methods defined in a specific class.
    Takes inputs: {'class_name': {'type': 'string', 'description': 'The name of the class to find methods for.'}, 'file_path': {'type': 'string', 'nullable': True, 'description': 'Optional file path to limit the search to a specific file.'}}
    Returns an output of type: array
- get_file_structure: Get the complete structure of a Python file including functions, classes, and variables.
    Takes inputs: {'file_path': {'type': 'string', 'description': 'The relative path to the file, including folder and filename.'}}
    Returns an output of type: object
- get_code_segment: Get a segment of code from a file between the specified line numbers.
    Takes inputs: {'file_path': {'type': 'string', 'description': 'The relative path to the file, including folder and filename.'}, 'start_line': {'type': 'integer', 'description': 'The starting line number (1-based).'}, 'end_line': {'type': 'integer', 'description': 'The ending line number (1-based, inclusive).'}}
    Returns an output of type: string
- generate_diff: Generate a unified diff between original and modified content.
    Takes inputs: {'original_content': {'type': 'string', 'description': 'Original file content'}, 'modified_content': {'type': 'string', 'description': 'Modified file content'}, 'file_path': {'type': 'string', 'description': 'Path to the file'}}
    Returns an output of type: string
- extract_method_context: Extract a method and its surrounding context from a file.
    Takes inputs: {'file_content': {'type': 'string', 'description': 'Content of the file'}, 'method_name': {'type': 'string', 'description': 'Name of the method to find'}}
    Returns an output of type: object
- identify_issue_patterns: Identify patterns in the file that may be causing the issue.
    Takes inputs: {'file_content': {'type': 'string', 'description': 'Content of the file'}, 'issue_description': {'type': 'string', 'description': 'Description of the GitHub issue'}}
    Returns an output of type: array
- apply_patch_to_content: Apply a series of patch operations to the file content.
    Takes inputs: {'original_content': {'type': 'string', 'description': 'Original file content'}, 'patch_operations': {'type': 'array', 'items': {'type': 'object', 'additionalProperties': {'type': 'any'}}, 'description': 'List of operations to apply, each being a dict with: - operation: "replace", "insert", or "delete" - start_line: Starting line number (1-indexed) - end_line: Ending line number for replace/delete (1-indexed) - content: New content for replace/insert operations'}}
    Returns an output of type: string
- final_answer: Provides a final answer to the given problem.
    Takes inputs: {'answer': {'type': 'any', 'description': 'The final answer to the problem'}}
    Returns an output of type: any

Here are the rules you should always follow to solve your task:
1. Always provide a 'Thought:' sequence, and a 'Code:\n```py' sequence ending with '```<end_code>' sequence, else you will fail.
2. Use only variables that you have defined!
3. Always use the right arguments for the tools. DO NOT pass the arguments as a dict as in 'answer = wiki({'query': "What is the place where James Bond lives?"})', but use the arguments directly as in 'answer = wiki(query="What is the place where James Bond lives?")'.
4. Take care to not chain too many sequential tool calls in the same code block, especially when the output format is unpredictable. For instance, a call to search has an unpredictable return format, so do not have another tool call that depends on its output in the same block: rather output results with print() to use them in the next block.
5. Call a tool only when needed, and never re-do a tool call that you previously did with the exact same parameters.
6. Don't name any new variable with the same name as a tool: for instance don't name a variable 'final_answer'.
7. Never create any notional variables in our code, as having these in your logs will derail you from the true variables.
8. You can use imports in your code, but only from the following list of modules: ['collections', 're', 'time', 'itertools', 'math', 'stat', 'statistics', 'datetime', 'random', 'unicodedata', 'queue']
9. The state persists between code executions: so if in one step you've created variables or imported modules, these will all persist.
10. Don't give up! You're in charge of solving the task, not providing directions to solve it.

Now Begin! Analyze the issue and file, develop a patch, and generate a diff to fix the problem.

In [7]:
patch_agent.run(task)

╭───────────────────────────────────────────── New run - patch_agent ─────────────────────────────────────────────╮
│                                                                                                                 │
│ You are tasked with understanding an issue in the following github repository pylint-dev/astroid.               │
│ You will be given a description of the issue and tools to search the codebase.                                  │
│ Your job is to determine, what file or files are causing the issue.                                             │
│ The tools provided will allow you to search the codebase and find the files that are causing the issue.         │
│ Determine what is the function or class where this issue is happening                                           │
│ ### Issue Description                                                                                           │
│ TypeError: unsupported format string passed to NoneType.__format__                                              │
│ Regression in #2459                                                                                             │
│                                                                                                                 │
│ ### Steps to reproduce                                                                                          │
│ a.py:                                                                                                           │
│ ```py                                                                                                           │
│ class A:                                                                                                        │
│     def __init__(self):                                                                                         │
│         self._magnitude = None                                                                                  │
│                                                                                                                 │
│     def name(self) -> str | None:                                                                               │
│         if self._magnitude:                                                                                     │
│             return f"M {self._magnitude:.1f}"                                                                   │
│ ```                                                                                                             │
│ ```                                                                                                             │
│ pylint a.py                                                                                                     │
│ ```                                                                                                             │
│ ### Current behavior                                                                                            │
│ ```                                                                                                             │
│   File "/Users/jwalls/release/lib/python3.12/site-packages/astroid/nodes/node_classes.py", line 4778, in        │
│ _infer_from_values                                                                                              │
│     yield from nodes[0]._infer(context, **kwargs)                                                               │
│   File "/Users/jwalls/release/lib/python3.12/site-packages/astroid/nodes/node_classes.py", line 4695, in _infer │
│     formatted = format(value.value, format_spec.value)                                                          │
│                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^                                                          │
│ TypeError: unsupported format string passed to NoneType.__format__                                              │
│ ```                                                   

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  file_content = get_file_content(file_path="astroid/nodes/node_classes.py")                                       
  print(file_content)                                                                                              
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
# Licensed under the LGPL: https://www.gnu.org/licenses/old-licenses/lgpl-2.1.en.html
# For details: https://github.com/pylint-dev/astroid/blob/main/LICENSE
# Copyright (c) https://github.com/pylint-dev/astroid/blob/main/CONTRIBUTORS.txt

"""Module for some node classes. More nodes in scoped_nodes.py"""

from __future__ import annotations

import abc
import ast
import itertools
import operator
import sys
import typing
import warnings
from collections.abc import Callable, Generator, Iterable, Iterator, Mapping
from functools import cached_property
from typing import (
    TYPE_CHECKING,
    Any,
    ClassVar,
    Literal,
    Optional,
    Union,
)

from astroid import decorators, protocols, util
from astroid.bases import Instance, _infer_stmts
from astroid.const import _EMPTY_OBJECT_MARKER, Context
from astroid.context import CallContext, InferenceContext, copy_context
from astroid.exceptions import (
    AstroidBuildingError,
    AstroidError,
    AstroidIndexError,
    AstroidTypeError,
    AstroidValueError,
    AttributeInferenceError,
    InferenceError,
    NameInferenceError,
    NoDefault,
    ParentMissingError,
    _NonDeducibleTypeHierarchy,
)
from astroid.interpreter import dunder_lookup
from astroid.manager import AstroidManager
from astroid.nodes import _base_nodes
from astroid.nodes.const import OP_PRECEDENCE
from astroid.nodes.node_ng import NodeNG
from astroid.typing import (
    ConstFactoryResult,
    InferenceErrorInfo,
    InferenceResult,
    SuccessfulInferenceResult,
)

if sys.version_info >= (3, 11):
    from typing import Self
else:
    from typing_extensions import Self

if TYPE_CHECKING:
    from astroid import nodes
    from astroid.nodes import LocalsDictNodeNG


def _is_const(value) -> bool:
    return isinstance(value, tuple(CONST_CLS))


_NodesT = typing.TypeVar("_NodesT", bound=NodeNG)
_BadOpMessageT = typing.TypeVar("_BadOpMessageT", bound=util.BadOperationMessage)

AssignedStmtsPossibleNode = Union["List", "Tuple", "AssignName", "AssignAttr", None]
AssignedStmtsCall = Callable[
    [
        _NodesT,
        AssignedStmtsPossibleNode,
        Optional[InferenceContext],
        Optional[list[int]],
    ],
    Any,
]
InferBinaryOperation = Callable[
    [_NodesT, Optional[InferenceContext]],
    Generator[Union[InferenceResult, _BadOpMessageT], None, None],
]
InferLHS = Callable[
    [_NodesT, Optional[InferenceContext]],
    Generator[InferenceResult, None, Optional[InferenceErrorInfo]],
]
InferUnaryOp = Callable[[_NodesT, str], ConstFactoryResult]


@decorators.raise_if_nothing_inferred
def unpack_infer(stmt, context: InferenceContext | None = None):
    """recursively generate nodes inferred by the given statement.
    If the inferred value is a list or a tuple, recurse on the elements
    """
    if isinstance(stmt, (List, Tuple)):
        for elt in stmt.elts:
            if elt is util.Uninferable:
                yield elt
                continue
            yield from unpack_infer(elt, context)
        return {"node": stmt, "context": context}
    # if inferred is a final node, return it and stop
    inferred = next(stmt.infer(context), util.Uninferable)
    if inferred is stmt:
        yield inferred
        return {"node": stmt, "context": context}
    # else, infer recursively, except Uninferable object that should be returned as is
    for inferred in stmt.infer(context):
        if isinstance(inferred, util.UninferableBase):
            yield inferred
        else:
            yield from unpack_infer(inferred, context)

    return {"node": stmt, "context": context}


def are_exclusive(stmt1, stmt2, exceptions: list[str] | None = None) -> bool:
    """return true if the two given statements are mutually exclusive

    `exceptions` may be a list of exception names. If specified, discard If
    branches and check one of the statement is in an exception handler catching
    one of the given exceptions.

    algorithm :
     1) index stmt1's parents
     2) climb amon

[Step 0: Duration 36.35 seconds| Input tokens: 4,241 | Output tokens: 204]

...

Execution logs:
--- a/astroid/nodes/node_classes.py
+++ b/astroid/nodes/node_classes.py
@@ -4692,6 +4692,21 @@
                         yield util.Uninferable
                         uninferable_already_generated = True
                     continue
+                if not isinstance(value, Const):
+                    if not uninferable_already_generated:
+                        yield util.Uninferable
+                        uninferable_already_generated = True
+                    continue
+                if not isinstance(format_spec, Const):
+                    if not uninferable_already_generated:
+                        yield util.Uninferable
+                        uninferable_already_generated = True
+                    continue
+                if value.value is None or format_spec.value is None:
+                    if not uninferable_already_generated:
+                        yield util.Uninferable
+                        uninferable_already_generated = True
+                    continue
                 formatted = format(value.value, format_spec.value)
                 yield Const(
                     formatted,
@@ -5536,4 +5551,4 @@
         )
         instance.postinit(_create_dict_items(value, instance))
         return instance
-    return Const(value)
+    return Const(value)

Out: None

[Step 5: Duration 4.85 seconds| Input tokens: 98,804 | Output tokens: 1,656]

Reached max steps.

[Step 6: Duration 4.85 seconds| Input tokens: 116,169 | Output tokens: 2,046]

'The issue is happening in the file `astroid/nodes/node_classes.py` in the `FormattedValue._infer` function, specifically at line 4695. The error is a `TypeError: unsupported format string passed to NoneType.__format__`, which occurs when the `format` function is called with a `None` value and a format specifier. To fix this, a check should be added to ensure that both `value.value` and `format_spec.value` are not `None` before calling the `format` function. If either of them is `None`, `util.Uninferable` should be yielded.\n```diff\n--- a/astroid/nodes/node_classes.py\n+++ b/astroid/nodes/node_classes.py\n@@ -4692,6 +4692,21 @@\n                         yield util.Uninferable\n                         uninferable_already_generated = True\n                     continue\n+                if not isinstance(value, Const):\n+                    if not uninferable_already_generated:\n+                        yield util.Uninferable\n+                        uninferable_already_generated = Tr

In [8]:
print(issue_data.loc[0]['patch'])

diff --git a/ChangeLog b/ChangeLog
index 4560e5d2b7..c08b1cbf2c 100644
--- a/ChangeLog
+++ b/ChangeLog
@@ -13,6 +13,9 @@ What's New in astroid 3.3.1?
 Release date: TBA
 
+* Fix a crash introduced in 3.3.0 involving invalid format strings.
+
+  Closes #2492
 
 
 What's New in astroid 3.3.0?
diff --git a/astroid/nodes/node_classes.py b/astroid/nodes/node_classes.py
index c1c7af36da..1924c78eba 100644
--- a/astroid/nodes/node_classes.py
+++ b/astroid/nodes/node_classes.py
@@ -4687,19 +4687,24 @@ def _infer(
                     uninferable_already_generated = True
                 continue
             for value in self.value.infer(context, **kwargs):
-                if not isinstance(value, Const):
-                    if not uninferable_already_generated:
-                        yield util.Uninferable
-                        uninferable_already_generated = True
-                    continue
-                formatted = format(value.value, format_spec.value)
-                yield Co

In [9]:
if os.path.exists('repository.db'):
    os.remove('repository.db')


In [10]:
repo_path

'../data/repos/repo__pylint-dev__astroid-2496'

In [11]:
parse_repository(repo_path)
analyze_python_files()

Repository parsing complete. Processed 313 files.
Syntax error in file tests/testdata/python3/data/unicode_package/__init__.py: invalid non-printable character U+FEFF (<unknown>, line 1)
Python file analysis complete. Results:
  total_python_files: 258
  total_functions: 3104
  total_classes: 403
  total_variables: 6012
  files_with_parse_errors: 1


{'total_python_files': 258,
 'total_functions': 3104,
 'total_classes': 403,
 'total_variables': 6012,
 'files_with_parse_errors': 1}

In [12]:

SYSTEM_PROMPT = """You are an expert assistant who can solve any task using code blobs.
You will be given a task to solve as best you can.
To do so, you have been given access to a list of tools: these tools are basically Python functions which you can call with code. To solve the task, you must plan forward to proceed in a series of steps, in a cycle of 'Thought:', 'Code:', and 'Observation:' sequences.
At each step, in the 'Thought:' sequence, you should first explain your reasoning towards solving the task and the tools that you want to use. Then in the 'Code:' sequence, you should write the code in simple Python. The code sequence must end with '<end_code>' sequence.
During each intermediate step, you can use 'print()' to save whatever important information you will then need.
These print outputs will then appear in the 'Observation:' field, which will be available as input for the next step.
In the end you have to return a final answer using the final_answer tool."""

In [13]:

model = create_model()

analysis_agent = create_analysis_agent(model=model)

analysis_agent.visualize()

CodeAgent | gemini-2.0-flash
├── ✅ Authorized imports: []
└── 🛠️ Tools:
    ┏━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
    ┃ Name                   ┃ Description                              ┃ Arguments                               ┃
    ┡━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
    │ get_folders            │ Get all unique folder paths in the       │                                         │
    │                        │ repository.                              │                                         │
    │ get_files_in_folder    │ Get all files in a specific folder.      │ folder_path (`string`): The relative    │
    │                        │                                          │ folder path to query.                   │
    │ get_files_by_extension │ Get all files with a specific extension. │ extension (`string`): The file          │
    │                        │                                          │ extension to filter by (without the     │
    │                        │                                          │ dot, e.g., 'py' not '.py').             │
    │ get_file_by_path       │ Get file information for a specific file │ file_path (`string`): The relative path │
    │                        │ path.                                    │ to the file, including folder and       │
    │                        │                                          │ filename.                               │
    │ get_file_content       │ Get the content of a file by path.       │ file_path (`string`): The relative path │
    │                        │                                          │ to the file, including folder and       │
    │                        │                                          │ filename.                               │
    │ get_entity_at_line     │ Find the code entity (function, class,   │ file_path (`string`): The relative path │
    │                        │ or variable) at a specific line in a     │ to the file, including folder and       │
    │                        │ file.                                    │ filename.                               │
    │                        │                                          │ line_number (`integer`): The line       │
    │                        │                                          │ number to check.                        │
    │ get_function_by_name   │ Find functions or methods by name,       │ function_name (`string`): The name of   │
    │                        │ optionally filtering by class and file.  │ the function or method to find.         │
    │                        │                                          │ class_name (`string`): Optional class   │
    │                        │                                          │ name to filter by (for methods).        │
    │                        │                                          │ file_path (`string`): Optional file     │
    │                        │                                          │ path to limit the search to a specific  │
    │                        │                                          │ file.                                   │
    │ get_class_by_name      │ Find classes by name, optionally         │ class_name (`string`): The name of the  │
    │                        │ filtering by file.                       │ class to find.                          │
    │                        │                                          │ file_path (`string`): Optional file     │
    │                        │                                          │ path to limit the search to a specific  │
    │                        │                                          │ file.                                   │
    │ get_class_methods      │ Get all methods defined in a specific    │ class_name (`string`): The

In [14]:
# llm_model_pth = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"
agent = create_code_agent(model=model, analysis_agent=analysis_agent)

In [15]:
agent.visualize()

CodeAgent | gemini-2.0-flash
├── ✅ Authorized imports: []
├── 🛠️ Tools:
│   ┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
│   ┃ Name         ┃ Description                                   ┃ Arguments                                    ┃
│   ┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   │ final_answer │ Provides a final answer to the given problem. │ answer (`any`): The final answer to the      │
│   │              │                                               │ problem                                      │
│   └──────────────┴───────────────────────────────────────────────┴──────────────────────────────────────────────┘
└── 🤖 Managed agents:
    └── analysis_agent | CodeAgent | gemini-2.0-flash
        ├── ✅ Authorized imports: []
        ├── 📝 Description: This agent is responsible for analyzing the codebase and determining what files are 
        │   causing the issue.
        └── 🛠️ Tools:
            ┏━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
            ┃ Name                   ┃ Description                          ┃ Arguments                           ┃
            ┡━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
            │ get_folders            │ Get all unique folder paths in the   │                                     │
            │                        │ repository.                          │                                     │
            │ get_files_in_folder    │ Get all files in a specific folder.  │ folder_path (`string`): The         │
            │                        │                                      │ relative folder path to query.      │
            │ get_files_by_extension │ Get all files with a specific        │ extension (`string`): The file      │
            │                        │ extension.                           │ extension to filter by (without the │
            │                        │                                      │ dot, e.g., 'py' not '.py').         │
            │ get_file_by_path       │ Get file information for a specific  │ file_path (`string`): The relative  │
            │                        │ file path.                           │ path to the file, including folder  │
            │                        │                                      │ and filename.                       │
            │ get_file_content       │ Get the content of a file by path.   │ file_path (`string`): The relative  │
            │                        │                                      │ path to the file, including folder  │
            │                        │                                      │ and filename.                       │
            │ get_entity_at_line     │ Find the code entity (function,      │ file_path (`string`): The relative  │
            │                        │ class, or variable) at a specific    │ path to the file, including folder  │
            │                        │ line in a file.                      │ and filename.                       │
            │                        │                                      │ line_number (`integer`): The line   │
            │                        │                                      │ number to check.                    │
            │ get_function_by_name   │ Find functions or methods by name,   │ function_name (`string`): The name  │
            │                        │ optionally filtering by class and    │ of the function or method to find.  │
            │                        │ file.                                │ class_name (`string`): Optional     │
            │                        │                                      │ class name to filter by (for        │
            │                        │                

In [16]:
Markdown(agent.system_prompt)

You are an expert software engineer assistant who can solve GitHub issues and navigate codebases using code blobs. You will be given a task related to a code repository to solve as best you can.

To do so, you have been given access to a list of tools: these tools are Python functions that help you explore and understand the repository structure, examine code files, and analyze code entities like functions and classes.

To solve the task, you must plan forward to proceed in a series of steps, in a cycle of 'Thought:', 'Code:', and 'Observation:' sequences.

At each step, in the 'Thought:' sequence, you should first explain your reasoning towards solving the issue and the tools that you want to use.
Then in the 'Code:' sequence, you should write the code in simple Python. The code sequence must end with '<end_code>' sequence.
During each intermediate step, you can use 'print()' to save whatever important information you will then need.
These print outputs will then appear in the 'Observation:' field, which will be available as input for the next step.
In the end you have to return a final answer using the `final_answer` tool.

Here are a few examples using the repository exploration tools:

---
Task: "Find all Python files in the repository and identify the largest one by line count."

Thought: I will solve this in multiple steps. First, I'll explore what files with .py extension exist in the repository using the `get_files_by_extension` tool, which will list all Python files. Then I'll determine which one has the highest line count.
Code:
```py
# First, let's get all Python files in the repository
python_files = get_files_by_extension(extension="py")
print(f"Found {len(python_files)} Python files:")
for file in python_files:
    print(f"{file.full_path}: {file.line_count} lines")
```<end_code>
Observation:
Found 5 Python files:
src/main.py: 89 lines
src/utils/helpers.py: 56 lines
src/models/user.py: 127 lines
src/models/product.py: 142 lines
src/api/routes.py: 218 lines

Thought: Now I can identify the largest Python file by sorting them based on line count. Looking at the observation, I can already see that src/api/routes.py has the most lines (218), but let's confirm programmatically.
Code:
```py
# Find the file with the maximum line count
largest_file = max(python_files, key=lambda file: file.line_count)
print(f"The largest Python file is: {largest_file.full_path}")
print(f"It has {largest_file.line_count} lines")

# Let's also get a quick preview of what this file contains
file_structure = get_file_structure(file_path=largest_file.full_path)
print(f"This file contains:")
print(f"- {len(file_structure['classes'])} classes")
print(f"- {len(file_structure['functions'])} functions")
print(f"- {len(file_structure['variables'])} variables")
```<end_code>
Observation:
The largest Python file is: src/api/routes.py
It has 218 lines
This file contains:
- 2 classes
- 12 functions
- 8 variables

Thought: I have now confirmed that src/api/routes.py is the largest Python file in the repository with 218 lines. It contains 2 classes, 12 functions, and 8 variables.
Code:
```py
final_answer("The largest Python file in the repository is src/api/routes.py with 218 lines. This file defines 2 classes, 12 functions, and 8 variables, suggesting it handles multiple API route definitions.")
```<end_code>

---
Task: "What is the result of the following operation: 5 + 3 + 1294.678?"

Thought: I will use python code to compute the result of the operation and then return the final answer using the `final_answer` tool
Code:
```py
result = 5 + 3 + 1294.678
final_answer(result)
```<end_code>

---
Task:
"Answer the question in the variable `question` about the image stored in the variable `image`. The question is in French.
You have been provided with these additional arguments, that you can access using the keys as variables in your python code:
{'question': 'Quel est l'animal sur l'image?', 'image': 'path/to/image.jpg'}"

Thought: I will use the following tools: `translator` to translate the question into English and then `image_qa` to answer the question on the input image.
Code:
```py
translated_question = translator(question=question, src_lang="French", tgt_lang="English")
print(f"The translated question is {translated_question}.")
answer = image_qa(image=image, question=translated_question)
final_answer(f"The answer is {answer}")
```<end_code>

---
Task:
In a 1979 interview, Stanislaus Ulam discusses with Martin Sherwin about other great physicists of his time, including Oppenheimer.
What does he say was the consequence of Einstein learning too much math on his creativity, in one word?

Thought: I need to find and read the 1979 interview of Stanislaus Ulam with Martin Sherwin.
Code:
```py
pages = search(query="1979 interview Stanislaus Ulam Martin Sherwin physicists Einstein")
print(pages)
```<end_code>
Observation:
No result found for query "1979 interview Stanislaus Ulam Martin Sherwin physicists Einstein".

Thought: The query was maybe too restrictive and did not find any results. Let's try again with a broader query.
Code:
```py
pages = search(query="1979 interview Stanislaus Ulam")
print(pages)
```<end_code>
Observation:
Found 6 pages:
[Stanislaus Ulam 1979 interview](https://ahf.nuclearmuseum.org/voices/oral-histories/stanislaus-ulams-interview-1979/)

[Ulam discusses Manhattan Project](https://ahf.nuclearmuseum.org/manhattan-project/ulam-manhattan-project/)

(truncated)

Thought: I will read the first 2 pages to know more.
Code:
```py
for url in ["https://ahf.nuclearmuseum.org/voices/oral-histories/stanislaus-ulams-interview-1979/", "https://ahf.nuclearmuseum.org/manhattan-project/ulam-manhattan-project/"]:
    whole_page = visit_webpage(url)
    print(whole_page)
    print("\n" + "="*80 + "\n")  # Print separator between pages
```<end_code>
Observation:
Manhattan Project Locations:
Los Alamos, NM
Stanislaus Ulam was a Polish-American mathematician. He worked on the Manhattan Project at Los Alamos and later helped design the hydrogen bomb. In this interview, he discusses his work at
(truncated)

Thought: I now have the final answer: from the webpages visited, Stanislaus Ulam says of Einstein: "He learned too much mathematics and sort of diminished, it seems to me personally, it seems to me his purely physics creativity." Let's answer in one word.
Code:
```py
final_answer("diminished")
```<end_code>

---
Task: "Which city has the highest population: Guangzhou or Shanghai?"

Thought: I need to get the populations for both cities and compare them: I will use the tool `search` to get the population of both cities.
Code:
```py
for city in ["Guangzhou", "Shanghai"]:
    print(f"Population {city}:", search(f"{city} population")
```<end_code>
Observation:
Population Guangzhou: ['Guangzhou has a population of 15 million inhabitants as of 2021.']
Population Shanghai: '26 million (2019)'

Thought: Now I know that Shanghai has the highest population.
Code:
```py
final_answer("Shanghai")
```<end_code>

---
Task: "What is the current age of the pope, raised to the power 0.36?"

Thought: I will use the tool `wiki` to get the age of the pope, and confirm that with a web search.
Code:
```py
pope_age_wiki = wiki(query="current pope age")
print("Pope age as per wikipedia:", pope_age_wiki)
pope_age_search = web_search(query="current pope age")
print("Pope age as per google search:", pope_age_search)
```<end_code>
Observation:
Pope age: "The pope Francis is currently 88 years old."

Thought: I know that the pope is 88 years old. Let's compute the result using python code.
Code:
```py
pope_current_age = 88 ** 0.36
final_answer(pope_current_age)
```<end_code>

Above example were using notional tools that might not exist for you. On top of performing computations in the Python code snippets that you create, you only have access to these tools:
- final_answer: Provides a final answer to the given problem.
    Takes inputs: {'answer': {'type': 'any', 'description': 'The final answer to the problem'}}
    Returns an output of type: any
You can also give tasks to team members.
Calling a team member works the same as for calling a tool: simply, the only argument you can give in the call is 'task', a long string explaining your task.
Given that this team member is a real human, you should be very verbose in your task.
Here is a list of the team members that you can call:
- analysis_agent: This agent is responsible for analyzing the codebase and determining what files are causing the issue.

Here are the rules you should always follow to solve your task:
1. Always provide a 'Thought:' sequence, and a 'Code:\n```py' sequence ending with '```<end_code>' sequence, else you will fail.
2. Use only variables that you have defined!
3. Always use the right arguments for the tools. DO NOT pass the arguments as a dict as in 'answer = wiki({'query': "What is the place where James Bond lives?"})', but use the arguments directly as in 'answer = wiki(query="What is the place where James Bond lives?")'.
4. Take care to not chain too many sequential tool calls in the same code block, especially when the output format is unpredictable. For instance, a call to search has an unpredictable return format, so do not have another tool call that depends on its output in the same block: rather output results with print() to use them in the next block.
5. Call a tool only when needed, and never re-do a tool call that you previously did with the exact same parameters.
6. Don't name any new variable with the same name as a tool: for instance don't name a variable 'final_answer'.
7. Never create any notional variables in our code, as having these in your logs will derail you from the true variables.
8. You can use imports in your code, but only from the following list of modules: ['collections', 're', 'time', 'itertools', 'math', 'stat', 'statistics', 'datetime', 'random', 'unicodedata', 'queue']
9. The state persists between code executions: so if in one step you've created variables or imported modules, these will all persist.
10. Don't give up! You're in charge of solving the task, not providing directions to solve it.

Now Begin! If you solve the task correctly, you will receive a reward of $1,000,000.

In [17]:
row

instance_id                                   pylint-dev__astroid-2496
repo                                                pylint-dev/astroid
problem_statement    TypeError: unsupported format string passed to...
patch                diff --git a/ChangeLog b/ChangeLog\nindex 4560...
test_patch           diff --git a/tests/test_inference.py b/tests/t...
pull_number                                                       2496
base_commit                   8d3cdbbe6685fd8cf211816bec56c90f38f1859e
PASS_TO_PASS         [tests/test_inference.py::InferenceUtilsTest::...
FAIL_TO_PASS         [tests/test_inference.py::test_formatted_fstri...
issue_numbers                                                   [2492]
Name: 0, dtype: object

In [18]:
task = f"""You are tasked with understanding an issue in the following github repository {row['repo']}.
You will be given a description of the issue and tools to search the codebase.
Your job is to determine, what file or files are causing the issue.
The tools provided will allow you to search the codebase and find the files that are causing the issue.
Determine what is the function or class where this issue is happening
### Issue Description
{row['problem_statement']}"""

In [19]:
agent.run(task)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You are tasked with understanding an issue in the following github repository pylint-dev/astroid.               │
│ You will be given a description of the issue and tools to search the codebase.                                  │
│ Your job is to determine, what file or files are causing the issue.                                             │
│ The tools provided will allow you to search the codebase and find the files that are causing the issue.         │
│ Determine what is the function or class where this issue is happening                                           │
│ ### Issue Description                                                                                           │
│ TypeError: unsupported format string passed to NoneType.__format__                                              │
│ Regression in #2459                                                                                             │
│                                                                                                                 │
│ ### Steps to reproduce                                                                                          │
│ a.py:                                                                                                           │
│ ```py                                                                                                           │
│ class A:                                                                                                        │
│     def __init__(self):                                                                                         │
│         self._magnitude = None                                                                                  │
│                                                                                                                 │
│     def name(self) -> str | None:                                                                               │
│         if self._magnitude:                                                                                     │
│             return f"M {self._magnitude:.1f}"                                                                   │
│ ```                                                                                                             │
│ ```                                                                                                             │
│ pylint a.py                                                                                                     │
│ ```                                                                                                             │
│ ### Current behavior                                                                                            │
│ ```                                                                                                             │
│   File "/Users/jwalls/release/lib/python3.12/site-packages/astroid/nodes/node_classes.py", line 4778, in        │
│ _infer_from_values                                                                                              │
│     yield from nodes[0]._infer(context, **kwargs)                                                               │
│   File "/Users/jwalls/release/lib/python3.12/site-packages/astroid/nodes/node_classes.py", line 4695, in _infer │
│     formatted = format(value.value, format_spec.value)                                                          │
│                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^                                                          │
│ TypeError: unsupported format string passed to NoneType.__format__                                              │
│ ```                                                   

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  analysis_agent_task = """                                                                                        
  Analyze the astroid codebase to identify the file and function causing the following error:                      
  TypeError: unsupported format string passed to NoneType.__format__                                               
  The error occurs in `astroid/nodes/node_classes.py`, specifically in the `_infer_from_values` and `_infer`       
  methods as seen in the traceback below:                                                                          
                                                                                                                   
  The provided code snippet that triggers the error is:                                                            
                                                                                                                   
  The issue is a regression introduced in #2459. Focus on changes related to formatting and inference in that      
  pull request.                                                                                                    
  """                                                                                                              
  analysis_agent_response = analysis_agent(task=analysis_agent_task)                                               
  print(analysis_agent_response)                                                                                   
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

╭─────────────────────────────────────────── New run - analysis_agent ────────────────────────────────────────────╮
│                                                                                                                 │
│ You're a helpful agent named 'analysis_agent'.                                                                  │
│ You have been submitted this task by your manager.                                                              │
│ ---                                                                                                             │
│ Task:                                                                                                           │
│                                                                                                                 │
│ Analyze the astroid codebase to identify the file and function causing the following error:                     │
│ TypeError: unsupported format string passed to NoneType.__format__                                              │
│ The error occurs in `astroid/nodes/node_classes.py`, specifically in the `_infer_from_values` and `_infer`      │
│ methods as seen in the traceback below:                                                                         │
│                                                                                                                 │
│ The provided code snippet that triggers the error is:                                                           │
│                                                                                                                 │
│ The issue is a regression introduced in #2459. Focus on changes related to formatting and inference in that     │
│ pull request.                                                                                                   │
│                                                                                                                 │
│ ---                                                                                                             │
│ You're helping your manager solve a wider task: so make sure to not provide a one-line answer, but give as much │
│ information as possible to give them a clear understanding of the answer.                                       │
│                                                                                                                 │
│ Your final_answer WILL HAVE to contain these parts:                                                             │
│ ### 1. Task outcome (short version):                                                                            │
│ ### 2. Task outcome (extremely detailed version):                                                               │
│ ### 3. Additional context (if relevant):                                                                        │
│                                                                                                                 │
│ Put all these in your final_answer tool, everything that you do not pass as an argument to final_answer will be │
│ lost.                                                                                                           │
│ And even if your task resolution is not successful, please return as much context as possible, so that your     │
│ manager can act upon this feedback.                                                                             │
│                                                                                                                 │
╰─ OpenAIServerModel - gemini-2.0-flash ──────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("The issue occurs in the `_infer` method of the `FormattedValue` class in                           
  `astroid/nodes/node_classes.py`. A potential fix is to add a check in `FormattedValue._infer` to ensure that     
  neither the value nor the format specification is `None` before calling `format`, and to return                  
  `util.Uninferable` if either is `None`.")                                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: The issue occurs in the `_infer` method of the `FormattedValue` class in 
`astroid/nodes/node_classes.py`. A potential fix is to add a check in `FormattedValue._infer` to ensure that 
neither the value nor the format specification is `None` before calling `format`, and to return `util.Uninferable` 
if either is `None`.

[Step 1: Duration 2.59 seconds| Input tokens: 25,122 | Output tokens: 825]

'The issue occurs in the `_infer` method of the `FormattedValue` class in `astroid/nodes/node_classes.py`. A potential fix is to add a check in `FormattedValue._infer` to ensure that neither the value nor the format specification is `None` before calling `format`, and to return `util.Uninferable` if either is `None`.'